In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import importlib
import subprocess
import sys

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import catboost as cd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squares_error
from sklearn.preprocessing import LabelEncoder
import warnings
warings.filterwarnings('ignore')

In [11]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv')
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (750000, 12)
Test shape: (250000, 11)


In [13]:
#Ckecking missing values#
train.isnull().sum()


id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [ ]:
# Target Distribution #
plt.figure(figsize=(8, 5))
sns.histplot(train['Listening_Time_minutes'],kde=True)
plt.title('Listening Time Distribution')
plt.show()


In [20]:
#Filling missing values
for col in ['Episode_Length_minutes', 'Guest_Popularity_percentage']:
    median_val = train[col].median()
    train[col].fillna(median_val, inplace=True)
    test[col].fillna(median_val, inplace=True)

/tmp/ipykernel_31/2105655418.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(median_val, inplace=True)
/tmp/ipykernel_31/2105655418.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

In [ ]:
# lable encoding
cat_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
    # New Features
    train['popularity_ratio'] = (train['Host_Popularity_percentage'] + train['Guest_Popularity_percentage']) / 2
test['popularity_ratio'] = (test['Host_Popularity_percentage'] + test['Guest_Popularity_percentage']) / 2

train['length_per_ad'] = train['Episode_Length_minutes'] / (train['Number_of_Ads'] + 1)
test['length_per_ad'] = test['Episode_Length_minutes'] / (test['Number_of_Ads'] + 1)

#Features
features = [col for col in train.columns if col not in ['id', 'Episode_Title', 'Listening_Time_minutes']]
X = train[features]
y = train['Listening_Time_minutes']
X_test = test[features]

 Model Training with LightGBM and CatBoost + Blending

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=42)
lgb_oof = np.zeros(X.shape[0])
cat_oof = np.zeros(X.shape[0])
lgb_preds = np.zeros(X_test.shape[0])
cat_preds = np.zeros(X_test.shape[0])

for fold, (trn_idx, val_idx) in enumerate(folds.split(X)):
    X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

#lightGMB
lgb_model = lgb.LGBMRegressor(
        n_estimators=10000,
        learning_rate=0.01,
        max_depth=8,
        num_leaves=50,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    lgb_model.fit(
         X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
    )

    lgb_oof[val_idx] = lgb_model.predict(X_valid)
    lgb_preds += lgb_model.predict(X_test) / folds.n_splits

#lgb_model.save()

#catBoost

cat_model = cb.CatBoostRegressor(
        iterations=10000,
        learning_rate=0.01,
        depth=8,
        loss_function='RMSE',
        early_stopping_rounds=100,
        random_state=42,
        verbose=100
    )

    cat_model.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    cat_oof[val_idx] = cat_model.predict(X_valid)
    cat_preds += cat_model.predict(X_test) / folds.n_splits

Blending and Evaluation

In [ ]:
inal_oof = (lgb_oof + cat_oof) / 2
final_preds = (lgb_preds + cat_preds) / 2

print(f"\n✅ Final CV RMSE: {np.sqrt(mean_squared_error(y, final_oof)):.5f}")

Feature Importance (LightGBM)

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Important Features (LightGBM)')
plt.show()